In [1]:
import random
import pennylane as qml
from pennylane import numpy as np
from myutils import *

#Magic Command, so changes in myutils module are reloaded
%load_ext autoreload
%autoreload 1
%aimport myutils

In [2]:
# Setting seeds for reproducible results
np.random.seed(1337)
random.seed(1337)

# Loading the data

In [46]:
data=  Datasets.get_preprocessed_datasets("PCA")["PCA"]["6,9"]

#scale data 0, 2*np
data["x_train"] = Preprocessing.minmax_scaler(data["x_train"] , min = 0,max = 2 * np.pi)
data["x_test"] = Preprocessing.minmax_scaler(data["x_test"], min = 0,max = 2 * np.pi)

for type in ["y_train_binary","y_test_binary"]:
    quantum_convert = []
    for i in range(len(data[type])):
        if data[type][i] == 0:
            quantum_convert.append(np.array([0,1]))
        else:
            quantum_convert.append(np.array([1,0]))

    data[type+"_quantum"] = np.array(quantum_convert)

In [47]:
# sampling and splitting 

n = 50
split = 0.7

training_indices = random.sample(range(len(data["x_train"])), int(n*split))
test_indices =  random.sample(range(len(data["x_test"])), int(n*(1-split)))

data["x_train_subset"] = data["x_train"][training_indices]
data["y_train_binary_quantum_subset"] = data["y_train_binary_quantum"][training_indices]

data["x_test_subset"] = data["x_test"][test_indices]
data["y_test_binary_quantum_subset"] = data["y_test_binary_quantum"][test_indices]


print("train {}, test{}".format(len(data["x_train_subset"]),len(data["x_test_subset"])))

train 35, test15


In [48]:
%%time
qcircuit = QuatumCircuit()
hist = qcircuit.train(x_train=data["x_train_subset"],
y_train=data["y_train_binary_quantum_subset"],
x_test=data["x_test_subset"],
y_test=data["y_test_binary_quantum_subset"],
epochs=5)

epoch 0: train_cost:0.880 train_acc:0.571 test_cost:0.535  test_acc:0.933
epoch 1: train_cost:0.405 train_acc:0.886 test_cost:0.244  test_acc:0.933
epoch 2: train_cost:0.288 train_acc:0.943 test_cost:0.257  test_acc:0.933
epoch 3: train_cost:0.257 train_acc:0.943 test_cost:0.232  test_acc:1.000
epoch 4: train_cost:0.256 train_acc:0.971 test_cost:0.215  test_acc:1.000
CPU times: user 27.3 s, sys: 85.6 ms, total: 27.4 s
Wall time: 27.4 s


In [56]:
correct = 0
validation_data = np.delete(data["x_test"],test_indices,axis=0)
validation_labels = np.delete(data["y_test_binary_quantum"],test_indices,axis=0)

for i in range(len(validation_data)):
    prediction = qcircuit.predict(validation_data[i])
    
    if prediction == validation_labels[i].tolist():
        correct+=1

print("accuracy: {:.4f} auf {} Validation-Daten".format(correct/len(data["x_test"]), len(validation_data)))

accuracy: 0.9578 auf 1952 Validation-Daten
